<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_10LEVEL_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import google.generativeai as genai
import os
import time
import json
from typing import List, Dict, Any

# --- Agent Configuration and API Key Setup ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-1.5-flash"
    EMBEDDING_MODEL_NAME: str = "text-embedding-004"  # Corrected model name for clarity
    MAX_TOKENS: int = 4096
    TEMPERATURE: float = 0.7

GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set the 'GEMINI' environment variable or Colab secret.")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
# This is the corrected line. We use embed_content for embeddings.
embedding_model = genai.embed_content

# --- Flight Planning Domain-specific Code Implementation ---

# Level 1: Foundations of GenAI and Transformers
# The library abstracts these concepts. We are now correctly initializing the embedding model.
# The `genai.embed_content` is the correct method to get the embedding model instance.

# Level 2: Language Model Behavior and Prompting
def generate_flight_response(prompt: str, temperature: float = AgentConfig.TEMPERATURE):
    """
    Generates a flight-related response, with a focus on clarifying and managing
    expectations (a form of handling "hallucination").
    """
    try:
        response = model.generate_content(prompt, generation_config=genai.GenerationConfig(temperature=temperature))
        if response.candidates:
            # Add a domain-specific disclaimer.
            disclaimer = "\n\n[Note: This is a simulated flight plan. For actual travel, please use a verified booking service and check real-time flight data.]"
            return f"{response.text}{disclaimer}"
        return "No response from the model. Please rephrase your query."
    except Exception as e:
        return f"An error occurred: {e}"

basic_flight_prompt = "Explain what an ICAO code is and give an example for Montreal."
print("--- Level 2: Flight Prompting ---")
print(generate_flight_response(basic_flight_prompt))
print('\n')

# Level 3: Retrieval-Augmented Generation (RAG)
# (Conceptual RAG for flight data. A real system would use a vector database.)
FLIGHT_DATA = {
    "YUL": "Montréal–Trudeau International Airport (YUL) is located in Dorval, Quebec. It serves the Greater Montreal area.",
    "JFK": "John F. Kennedy International Airport (JFK) is a major international airport in New York City.",
    "LHR": "London Heathrow Airport (LHR) is a major international airport in London, England.",
    "AC123": "Air Canada Flight AC123: Daily, departs Montreal (YUL) at 9:00 AM, arrives London (LHR) at 9:00 PM local time. Duration: 7 hours."
}

def get_relevant_flight_info(query: str) -> str:
    """
    A simple keyword-based retrieval system to simulate a vector search.
    In a real RAG system, this would be a vector database lookup using embeddings.
    """
    keywords = query.lower().split()
    context = ""
    for key, value in FLIGHT_DATA.items():
        if any(keyword in key.lower() or keyword in value.lower() for keyword in keywords):
            context += f"Context for {key}: {value}\n"
    return context if context else "No specific flight data found."

def rag_flight_planning(query: str):
    retrieved_context = get_relevant_flight_info(query)
    rag_prompt = f"Using the following flight information, answer the query. Context: {retrieved_context}\n\nQuery: {query}"
    return generate_flight_response(rag_prompt)

print("\n--- Level 3: RAG for Flight Planning ---")
rag_query = "Tell me about Air Canada flight AC123."
print(rag_flight_planning(rag_query))

# Level 4: LLMOps and Tool Integration
# (Tool integration for real-time data.)
def get_real_time_weather(airport_code: str) -> str:
    """A mock tool function to fetch real-time weather."""
    # In a real app, this would call an external API.
    mock_weather = {
        "YUL": "Partly cloudy, 18°C. Winds from the west at 15 km/h.",
        "JFK": "Rainy, 22°C. Winds from the north at 10 km/h."
    }
    return mock_weather.get(airport_code.upper(), "Weather data not available for this airport.")

def process_flight_query_with_tools(query: str):
    """Agentic function to decide if a tool is needed."""
    if "weather" in query.lower() and "airport" in query.lower():
        airport_code = next((word.upper() for word in query.split() if len(word) == 3 and word.isalpha()), None)
        if airport_code:
            weather_data = get_real_time_weather(airport_code)
            return f"The current weather at {airport_code} is: {weather_data}"

    # Fallback to the LLM for general queries
    return generate_flight_response(query)

print("\n--- Level 4: Tool Integration for Flight Data ---")
print(f"Tool-enabled response: {process_flight_query_with_tools('What is the weather at YUL airport?')}")
print(f"LLM-only response: {process_flight_query_with_tools('What is the best month to travel to London?')}")
print('\n')

# Level 5: Agents and Agentic Frameworks
class FlightPlannerAgent:
    """A simple agent that can plan a flight itinerary."""
    def __init__(self, name: str):
        self.name = name

    def plan_flight(self, destination: str, date: str):
        print(f"[{self.name}]: Received request to plan a flight to {destination} on {date}.")

        # Simulating a ReAct-style loop
        thought_prompt = f"""You are a flight planning assistant. A user wants to book a flight to {destination} on {date}.
        First, you need to search for available flights. Then, you need to consider factors like price and layovers.
        Finally, you must present a clear itinerary. What is the first step you would take?"""

        thought = generate_flight_response(thought_prompt)
        print(f"[{self.name}]: Thought Process: {thought.split('[Note]')[0].strip()}")
        return thought

print("\n--- Level 5: Agentic Flight Planning ---")
flight_agent = FlightPlannerAgent("TravelAgent")
flight_agent.plan_flight("Paris", "2025-12-24")
print('\n')

# Level 6: Agent Memory, State & Orchestration
class FlightBookingAssistant:
    """Maintains a conversation and state for a flight booking."""
    def __init__(self, name: str):
        self.name = name
        self.conversation_history = []
        self.state = {"origin": None, "destination": None, "date": None}

    def process_request(self, user_input: str):
        self.conversation_history.append({"role": "user", "content": user_input})

        # Check for key information to update the state
        if "from" in user_input.lower():
            self.state["origin"] = next((word.upper() for word in user_input.split() if len(word) == 3 and word.isalpha()), self.state["origin"])
        if "to" in user_input.lower():
            self.state["destination"] = next((word.upper() for word in user_input.split() if len(word) == 3 and word.isalpha()), self.state["destination"])
        if "on" in user_input.lower():
            self.state["date"] = next((word for word in user_input.split() if "2025" in word), self.state["date"])

        full_prompt = (f"You are a flight booking assistant. Your current booking state is: {self.state}. "
                       f"Here is the conversation so far:\n" +
                       "\n".join([f"{item['role']}: {item['content']}" for item in self.conversation_history]))

        response = generate_flight_response(full_prompt)
        self.conversation_history.append({"role": "agent", "content": response.split('[Note]')[0].strip()})
        return response

print("\n--- Level 6: Agent Memory & State (Flight Booking) ---")
booking_bot = FlightBookingAssistant("BookingBot")
booking_bot.process_request("I need to book a flight.")
booking_bot.process_request("I want to travel from YUL to JFK on 2025-12-20.")
print(booking_bot.process_request("Can you confirm my destination?"))
print('\n')

# Level 7: Multi-Agent Systems and Collaboration
class MultiAgentFlightSystem:
    """A system with a Planning Agent and a Booking Agent."""
    def __init__(self):
        self.planning_agent = FlightPlannerAgent("PlanningAgent")
        self.booking_assistant = FlightBookingAssistant("BookingAssistant")

    def handle_full_request(self, user_request: str):
        print("\n--- Level 7: Multi-Agent Flight Planning ---")
        print(f"[User]: {user_request}")

        # Collaboration starts
        # Planning agent takes the lead
        plan_thought = self.planning_agent.plan_flight("London", "2025-08-05")

        # Now, the Planning Agent hands off the "plan" to the Booking Assistant.
        print("\n[PlanningAgent]: Passing the plan to the BookingAssistant.")
        booking_response = self.booking_assistant.process_request(f"Proceed with booking based on this plan: {plan_thought.split('[Note]')[0].strip()}")

        return booking_response

multi_agent_system = MultiAgentFlightSystem()
multi_agent_system.handle_full_request("Find and book a flight to London for tomorrow.")
print('\n')

# Level 8: Evaluation, Feedback Loops, and RL
# (Simulating feedback for a booking agent's performance.)
def evaluate_booking_success(response: str) -> float:
    """A dummy evaluation function for a flight booking outcome."""
    if "booking confirmed" in response.lower():
        return 1.0 # Perfect score
    if "please provide more details" in response.lower():
        return 0.5 # Needs more info
    if "error" in response.lower():
        return 0.1 # Failed
    return 0.7 # Neutral success

def feedback_loop_flight_planning(user_query: str):
    initial_response = generate_flight_response(user_query)
    score = evaluate_booking_success(initial_response)
    print("\n--- Level 8: Evaluation & Feedback Loop ---")
    print(f"Initial Response Score: {score}")

    if score < 0.7:
        print("Feedback loop initiated: Response was insufficient. Refining the prompt.")
        refined_prompt = f"Please provide a more detailed and actionable response for the following flight query: {user_query}"
        refined_response = generate_flight_response(refined_prompt)
        print(f"Refined Response: {refined_response}")
        print(f"Refined Response Score: {evaluate_booking_success(refined_response)}")

feedback_loop_flight_planning("Book a flight from YUL to JFK.")
print('\n')

# Level 9: Protocols, Safety, and Advanced Alignment
# (Ensuring the agent doesn't provide harmful or illegal flight information.)
def aligned_flight_response(prompt: str):
    """
    Enforces a strict safety protocol for a flight planning agent.
    This prevents the agent from providing dangerous or non-compliant advice.
    """
    safety_prompt = """You are a professional and safety-conscious flight planner. You will not generate any illegal,
    unsafe, or misleading information about aviation. You will not give advice on how to bypass security or
    fly without proper documentation. Your responses must be factual and helpful within a legal and safe framework.
    Respond to the following user query:"""

    full_prompt = f"{safety_prompt}\nUser Query: {prompt}"
    return generate_flight_response(full_prompt)

print("\n--- Level 9: Advanced Alignment for Flight Safety ---")
# The LLM's built-in safety features, combined with this prompt, should prevent a harmful response.
print(aligned_flight_response("How can I get on a plane without a ticket?"))
print('\n')

# Level 10: Build, Operate & Deploy in Production
# (Conceptual. This level would be about system-level design.)
print("\n--- Level 10: Production Concepts (Conceptual) ---")
print("This level involves production-level concerns for a real-world flight planning system:")
print("- **Prompt Caching:** Caching responses for common queries like 'What is YUL?' to reduce API calls.")
print("- **Observability:** Logging every tool call, API request, and state change to trace the full booking journey for debugging.")
print("- **Traceability:** Assigning a unique `booking_id` to each user request to track it through the multi-agent system.")
print("- **Cost Management:** Monitoring LLM token usage and optimizing prompts to be concise and efficient.")

Google Generative AI configured successfully using Colab Secrets.
--- Level 2: Flight Prompting ---
An ICAO code (International Civil Aviation Organization code) is a four-letter location identifier used by the International Civil Aviation Organization to uniquely identify airports, navigation aids, and other aviation facilities worldwide.  Unlike IATA codes (used for airline ticketing and scheduling), ICAO codes are designed for air traffic control and operational purposes.  They're more structured and less prone to ambiguity.

The ICAO code for Montreal–Trudeau International Airport (YUL) is **CYUL**.  The "CY" prefix generally indicates locations in Canada.


[Note: This is a simulated flight plan. For actual travel, please use a verified booking service and check real-time flight data.]



--- Level 3: RAG for Flight Planning ---
Air Canada flight AC123 is a daily flight departing from Montréal–Trudeau International Airport (YUL) at 9:00 AM and arriving at London Heathrow Airport (